In [1]:
import math
import pandas as pd
import numpy as np
import time
from datetime import datetime
import copy
import os
import random
from typing import Optional, Any, Union, Callable, Tuple

import pandas_ta as ta
import torch
from torch import nn
from torch import Tensor
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.functional import _in_projection_packed
from torch.nn.modules.linear import NonDynamicallyQuantizableLinear
from torch.nn.init import constant_, xavier_normal_, xavier_uniform_
from torch.utils.data import TensorDataset, DataLoader
from statsmodels.tsa.stattools import adfuller
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
from matplotlib import pyplot
import plotly.offline as pyo
from d2l import torch as d2l

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
df=pd.read_csv("final_combine.csv")
df['date'] = pd.to_datetime(df['date'], format = '%d/%m/%Y')
df

,date,etom_flow_total,etom_transactions_count_flow,etom_flow_mean,mtoe_flow_total,mtoe_transactions_count_flow,mtoe_flow_mean,exchange_addresses_count_inflow,exchange_addresses_count_outflow,exchange_inflow_total,...,taker_sell_volume,taker_buy_ratio,taker_sell_ratio,taker_buy_sell_ratio,exchange_outflow_total,exchange_outflow_top10,exchange_outflow_mean,exchange_outflow_mean_ma7,exchange_netflow_total,miner_netflow_total
0,2016-01-01,239.315793,117,2.045434,245.037136,51,4.804650,17861,8225,23829.684650,...,2.402890e+05,0.497378,0.502622,0.989567,14070.243829,6927.139591,3.288208,3.953170,9759.440821,1693.817533
1,2016-01-02,128.735315,93,1.384251,237.954298,62,3.837973,18755,9181,11099.867349,...,3.758640e+05,0.484778,0.515222,0.940912,12835.223622,5547.606191,2.982855,3.617690,-1735.356273,1303.000408
2,2016-01-03,213.788376,121,1.766846,151.075527,51,2.962265,20945,9515,13295.729379,...,7.306770e+05,0.400210,0.599790,0.667251,13628.660295,7168.301744,2.824593,3.520564,-332.930916,1386.907567
3,2016-01-04,445.146297,141,3.157066,345.013905,65,5.307906,18019,11310,21875.941857,...,4.021300e+05,0.496113,0.503887,0.984572,20912.163154,12059.816788,4.605189,3.638665,963.778703,3541.923439
4,2016-01-05,506.206706,122,4.149235,263.934619,76,3.472824,18473,10344,19891.738077,...,3.320030e+05,0.309386,0.690614,0.447987,20082.407381,11042.671085,4.144120,3.658875,-190.669303,-4772.476726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2087,2021-09-18,21.113824,51,0.413997,416.311514,313,1.330069,16682,17195,15446.717053,...,7.718920e+09,0.505457,0.494543,1.022068,15066.723016,9332.271968,5.903888,8.594778,379.994037,156.954898
2088,2021-09-19,40.032766,66,0.606557,366.559556,309,1.186277,14052,15320,13368.003124,...,8.447522e+09,0.487900,0.512100,0.952744,14704.220166,8309.616751,5.905309,8.678698,-1336.217043,-3.806675
2089,2021-09-20,275.415063,103,2.673933,304.137201,360,0.844826,23068,22064,40580.947165,...,2.214091e+10,0.482775,0.517225,0.933394,35425.347382,17209.380872,11.221206,8.442772,5155.599783,-85.108841
2090,2021-09-21,91.547305,108,0.847660,174.964510,351,0.498474,21263,19524,34547.689220,...,2.298150e+10,0.492107,0.507893,0.968920,32213.383830,17636.679925,10.181221,8.342631,2334.305389,-86.106321


In [4]:
df['returns'] = 100 * df.close.pct_change()
df['high_pct_change'] = 100 * df.high.pct_change()
df['low_pct_change'] = 100 * df.low.pct_change()
df

,date,etom_flow_total,etom_transactions_count_flow,etom_flow_mean,mtoe_flow_total,mtoe_transactions_count_flow,mtoe_flow_mean,exchange_addresses_count_inflow,exchange_addresses_count_outflow,exchange_inflow_total,...,taker_buy_sell_ratio,exchange_outflow_total,exchange_outflow_top10,exchange_outflow_mean,exchange_outflow_mean_ma7,exchange_netflow_total,miner_netflow_total,returns,high_pct_change,low_pct_change
0,2016-01-01,239.315793,117,2.045434,245.037136,51,4.804650,17861,8225,23829.684650,...,0.989567,14070.243829,6927.139591,3.288208,3.953170,9759.440821,1693.817533,NaN,NaN,NaN
1,2016-01-02,128.735315,93,1.384251,237.954298,62,3.837973,18755,9181,11099.867349,...,0.940912,12835.223622,5547.606191,2.982855,3.617690,-1735.356273,1303.000408,-0.154853,0.070086,0.874018
2,2016-01-03,213.788376,121,1.766846,151.075527,51,2.962265,20945,9515,13295.729379,...,0.667251,13628.660295,7168.301744,2.824593,3.520564,-332.930916,1386.907567,-0.750269,-0.386339,-1.919439
3,2016-01-04,445.146297,141,3.157066,345.013905,65,5.307906,18019,11310,21875.941857,...,0.984572,20912.163154,12059.816788,4.605189,3.638665,963.778703,3541.923439,0.698154,0.009180,1.238562
4,2016-01-05,506.206706,122,4.149235,263.934619,76,3.472824,18473,10344,19891.738077,...,0.447987,20082.407381,11042.671085,4.144120,3.658875,-190.669303,-4772.476726,0.038235,-0.065855,0.259893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2087,2021-09-18,21.113824,51,0.413997,416.311514,313,1.330069,16682,17195,15446.717053,...,1.022068,15066.723016,9332.271968,5.903888,8.594778,379.994037,156.954898,2.122852,1.361268,0.639252
2088,2021-09-19,40.032766,66,0.606557,366.559556,309,1.186277,14052,15320,13368.003124,...,0.952744,14704.220166,8309.616751,5.905309,8.678698,-1336.217043,-3.806675,-2.190376,-0.907229,-0.435480
2089,2021-09-20,275.415063,103,2.673933,304.137201,360,0.844826,23068,22064,40580.947165,...,0.933394,35425.347382,17209.380872,11.221206,8.442772,5155.599783,-85.108841,-8.966624,-2.132911,-9.259215
2090,2021-09-21,91.547305,108,0.847660,174.964510,351,0.498474,21263,19524,34547.689220,...,0.968920,32213.383830,17636.679925,10.181221,8.342631,2334.305389,-86.106321,-5.296888,-7.846105,-6.757733


In [5]:
df['HL_sprd'] = ((df.high - df.low) / df.close).apply(np.cbrt)
df['CO_sprd'] = (df.close - df.open) / df.open
df

,date,etom_flow_total,etom_transactions_count_flow,etom_flow_mean,mtoe_flow_total,mtoe_transactions_count_flow,mtoe_flow_mean,exchange_addresses_count_inflow,exchange_addresses_count_outflow,exchange_inflow_total,...,exchange_outflow_top10,exchange_outflow_mean,exchange_outflow_mean_ma7,exchange_netflow_total,miner_netflow_total,returns,high_pct_change,low_pct_change,HL_sprd,CO_sprd
0,2016-01-01,239.315793,117,2.045434,245.037136,51,4.804650,17861,8225,23829.684650,...,6927.139591,3.288208,3.953170,9759.440821,1693.817533,NaN,NaN,NaN,0.292472,0.011277
1,2016-01-02,128.735315,93,1.384251,237.954298,62,3.837973,18755,9181,11099.867349,...,5547.606191,2.982855,3.617690,-1735.356273,1303.000408,-0.154853,0.070086,0.874018,0.257968,-0.005073
2,2016-01-03,213.788376,121,1.766846,151.075527,51,2.962265,20945,9515,13295.729379,...,7168.301744,2.824593,3.520564,-332.930916,1386.907567,-0.750269,-0.386339,-1.919439,0.319309,-0.011390
3,2016-01-04,445.146297,141,3.157066,345.013905,65,5.307906,18019,11310,21875.941857,...,12059.816788,4.605189,3.638665,963.778703,3541.923439,0.698154,0.009180,1.238562,0.273074,0.008310
4,2016-01-05,506.206706,122,4.149235,263.934619,76,3.472824,18473,10344,19891.738077,...,11042.671085,4.144120,3.658875,-190.669303,-4772.476726,0.038235,-0.065855,0.259893,0.257792,0.001443
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2087,2021-09-18,21.113824,51,0.413997,416.311514,313,1.330069,16682,17195,15446.717053,...,9332.271968,5.903888,8.594778,379.994037,156.954898,2.122852,1.361268,0.639252,0.332954,0.021228
2088,2021-09-19,40.032766,66,0.606557,366.559556,309,1.186277,14052,15320,13368.003124,...,8309.616751,5.905309,8.678698,-1336.217043,-3.806675,-2.190376,-0.907229,-0.435480,0.319772,-0.021900
2089,2021-09-20,275.415063,103,2.673933,304.137201,360,0.844826,23068,22064,40580.947165,...,17209.380872,11.221206,8.442772,5155.599783,-85.108841,-8.966624,-2.132911,-9.259215,0.483109,-0.089742
2090,2021-09-21,91.547305,108,0.847660,174.964510,351,0.498474,21263,19524,34547.689220,...,17636.679925,10.181221,8.342631,2334.305389,-86.106321,-5.296888,-7.846105,-6.757733,0.461613,-0.052905


In [6]:
df['log_returns'] = np.log(df['close'] /df['close'].shift(1))
df['log_returns']

0            NaN
1      -0.001550
2      -0.007531
3       0.006957
4       0.000382
          ...   
2087    0.021006
2088   -0.022147
2089   -0.093944
2090   -0.054423
2091    0.067234
Name: log_returns, Length: 2092, dtype: float64

In [7]:
df['vol_current'] = df['log_returns'].rolling(window=2).std() * np.sqrt(365)
df

,date,etom_flow_total,etom_transactions_count_flow,etom_flow_mean,mtoe_flow_total,mtoe_transactions_count_flow,mtoe_flow_mean,exchange_addresses_count_inflow,exchange_addresses_count_outflow,exchange_inflow_total,...,exchange_outflow_mean_ma7,exchange_netflow_total,miner_netflow_total,returns,high_pct_change,low_pct_change,HL_sprd,CO_sprd,log_returns,vol_current
0,2016-01-01,239.315793,117,2.045434,245.037136,51,4.804650,17861,8225,23829.684650,...,3.953170,9759.440821,1693.817533,NaN,NaN,NaN,0.292472,0.011277,NaN,NaN
1,2016-01-02,128.735315,93,1.384251,237.954298,62,3.837973,18755,9181,11099.867349,...,3.617690,-1735.356273,1303.000408,-0.154853,0.070086,0.874018,0.257968,-0.005073,-0.001550,NaN
2,2016-01-03,213.788376,121,1.766846,151.075527,51,2.962265,20945,9515,13295.729379,...,3.520564,-332.930916,1386.907567,-0.750269,-0.386339,-1.919439,0.319309,-0.011390,-0.007531,0.080802
3,2016-01-04,445.146297,141,3.157066,345.013905,65,5.307906,18019,11310,21875.941857,...,3.638665,963.778703,3541.923439,0.698154,0.009180,1.238562,0.273074,0.008310,0.006957,0.195726
4,2016-01-05,506.206706,122,4.149235,263.934619,76,3.472824,18473,10344,19891.738077,...,3.658875,-190.669303,-4772.476726,0.038235,-0.065855,0.259893,0.257792,0.001443,0.000382,0.088823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2087,2021-09-18,21.113824,51,0.413997,416.311514,313,1.330069,16682,17195,15446.717053,...,8.594778,379.994037,156.954898,2.122852,1.361268,0.639252,0.332954,0.021228,0.021006,0.412789
2088,2021-09-19,40.032766,66,0.606557,366.559556,309,1.186277,14052,15320,13368.003124,...,8.678698,-1336.217043,-3.806675,-2.190376,-0.907229,-0.435480,0.319772,-0.021900,-0.022147,0.582972
2089,2021-09-20,275.415063,103,2.673933,304.137201,360,0.844826,23068,22064,40580.947165,...,8.442772,5155.599783,-85.108841,-8.966624,-2.132911,-9.259215,0.483109,-0.089742,-0.093944,0.969921
2090,2021-09-21,91.547305,108,0.847660,174.964510,351,0.498474,21263,19524,34547.689220,...,8.342631,2334.305389,-86.106321,-5.296888,-7.846105,-6.757733,0.461613,-0.052905,-0.054423,0.533894


In [8]:
df['vol_future']=df['vol_current'].shift(-1)
df

,date,etom_flow_total,etom_transactions_count_flow,etom_flow_mean,mtoe_flow_total,mtoe_transactions_count_flow,mtoe_flow_mean,exchange_addresses_count_inflow,exchange_addresses_count_outflow,exchange_inflow_total,...,exchange_netflow_total,miner_netflow_total,returns,high_pct_change,low_pct_change,HL_sprd,CO_sprd,log_returns,vol_current,vol_future
0,2016-01-01,239.315793,117,2.045434,245.037136,51,4.804650,17861,8225,23829.684650,...,9759.440821,1693.817533,NaN,NaN,NaN,0.292472,0.011277,NaN,NaN,NaN
1,2016-01-02,128.735315,93,1.384251,237.954298,62,3.837973,18755,9181,11099.867349,...,-1735.356273,1303.000408,-0.154853,0.070086,0.874018,0.257968,-0.005073,-0.001550,NaN,0.080802
2,2016-01-03,213.788376,121,1.766846,151.075527,51,2.962265,20945,9515,13295.729379,...,-332.930916,1386.907567,-0.750269,-0.386339,-1.919439,0.319309,-0.011390,-0.007531,0.080802,0.195726
3,2016-01-04,445.146297,141,3.157066,345.013905,65,5.307906,18019,11310,21875.941857,...,963.778703,3541.923439,0.698154,0.009180,1.238562,0.273074,0.008310,0.006957,0.195726,0.088823
4,2016-01-05,506.206706,122,4.149235,263.934619,76,3.472824,18473,10344,19891.738077,...,-190.669303,-4772.476726,0.038235,-0.065855,0.259893,0.257792,0.001443,0.000382,0.088823,0.139956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2087,2021-09-18,21.113824,51,0.413997,416.311514,313,1.330069,16682,17195,15446.717053,...,379.994037,156.954898,2.122852,1.361268,0.639252,0.332954,0.021228,0.021006,0.412789,0.582972
2088,2021-09-19,40.032766,66,0.606557,366.559556,309,1.186277,14052,15320,13368.003124,...,-1336.217043,-3.806675,-2.190376,-0.907229,-0.435480,0.319772,-0.021900,-0.022147,0.582972,0.969921
2089,2021-09-20,275.415063,103,2.673933,304.137201,360,0.844826,23068,22064,40580.947165,...,5155.599783,-85.108841,-8.966624,-2.132911,-9.259215,0.483109,-0.089742,-0.093944,0.969921,0.533894
2090,2021-09-21,91.547305,108,0.847660,174.964510,351,0.498474,21263,19524,34547.689220,...,2334.305389,-86.106321,-5.296888,-7.846105,-6.757733,0.461613,-0.052905,-0.054423,0.533894,1.643500


In [9]:
df['ema10'] = ta.ema(df["close"], length=10)
df

,date,etom_flow_total,etom_transactions_count_flow,etom_flow_mean,mtoe_flow_total,mtoe_transactions_count_flow,mtoe_flow_mean,exchange_addresses_count_inflow,exchange_addresses_count_outflow,exchange_inflow_total,...,miner_netflow_total,returns,high_pct_change,low_pct_change,HL_sprd,CO_sprd,log_returns,vol_current,vol_future,ema10
0,2016-01-01,239.315793,117,2.045434,245.037136,51,4.804650,17861,8225,23829.684650,...,1693.817533,NaN,NaN,NaN,0.292472,0.011277,NaN,NaN,NaN,NaN
1,2016-01-02,128.735315,93,1.384251,237.954298,62,3.837973,18755,9181,11099.867349,...,1303.000408,-0.154853,0.070086,0.874018,0.257968,-0.005073,-0.001550,NaN,0.080802,NaN
2,2016-01-03,213.788376,121,1.766846,151.075527,51,2.962265,20945,9515,13295.729379,...,1386.907567,-0.750269,-0.386339,-1.919439,0.319309,-0.011390,-0.007531,0.080802,0.195726,NaN
3,2016-01-04,445.146297,141,3.157066,345.013905,65,5.307906,18019,11310,21875.941857,...,3541.923439,0.698154,0.009180,1.238562,0.273074,0.008310,0.006957,0.195726,0.088823,NaN
4,2016-01-05,506.206706,122,4.149235,263.934619,76,3.472824,18473,10344,19891.738077,...,-4772.476726,0.038235,-0.065855,0.259893,0.257792,0.001443,0.000382,0.088823,0.139956,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2087,2021-09-18,21.113824,51,0.413997,416.311514,313,1.330069,16682,17195,15446.717053,...,156.954898,2.122852,1.361268,0.639252,0.332954,0.021228,0.021006,0.412789,0.582972,47373.398942
2088,2021-09-19,40.032766,66,0.606557,366.559556,309,1.186277,14052,15320,13368.003124,...,-3.806675,-2.190376,-0.907229,-0.435480,0.319772,-0.021900,-0.022147,0.582972,0.969921,47350.740700
2089,2021-09-20,275.415063,103,2.673933,304.137201,360,0.844826,23068,22064,40580.947165,...,-85.108841,-8.966624,-2.132911,-9.259215,0.483109,-0.089742,-0.093944,0.969921,0.533894,46561.907572
2090,2021-09-21,91.547305,108,0.847660,174.964510,351,0.498474,21263,19524,34547.689220,...,-86.106321,-5.296888,-7.846105,-6.757733,0.461613,-0.052905,-0.054423,0.533894,1.643500,45502.261187


In [10]:
df = df.set_index(['date'])
#df.index = pd.to_datetime(df.index)
df

,etom_flow_total,etom_transactions_count_flow,etom_flow_mean,mtoe_flow_total,mtoe_transactions_count_flow,mtoe_flow_mean,exchange_addresses_count_inflow,exchange_addresses_count_outflow,exchange_inflow_total,exchange_inflow_top10,...,miner_netflow_total,returns,high_pct_change,low_pct_change,HL_sprd,CO_sprd,log_returns,vol_current,vol_future,ema10
date,,,,,,,,,,,,,,,,,,,,,
2016-01-01,239.315793,117,2.045434,245.037136,51,4.804650,17861,8225,23829.684650,15442.076341,...,1693.817533,NaN,NaN,NaN,0.292472,0.011277,NaN,NaN,NaN,NaN
2016-01-02,128.735315,93,1.384251,237.954298,62,3.837973,18755,9181,11099.867349,5736.771814,...,1303.000408,-0.154853,0.070086,0.874018,0.257968,-0.005073,-0.001550,NaN,0.080802,NaN
2016-01-03,213.788376,121,1.766846,151.075527,51,2.962265,20945,9515,13295.729379,7174.527336,...,1386.907567,-0.750269,-0.386339,-1.919439,0.319309,-0.011390,-0.007531,0.080802,0.195726,NaN
2016-01-04,445.146297,141,3.157066,345.013905,65,5.307906,18019,11310,21875.941857,11665.436371,...,3541.923439,0.698154,0.009180,1.238562,0.273074,0.008310,0.006957,0.195726,0.088823,NaN
2016-01-05,506.206706,122,4.149235,263.934619,76,3.472824,18473,10344,19891.738077,11643.719982,...,-4772.476726,0.038235,-0.065855,0.259893,0.257792,0.001443,0.000382,0.088823,0.139956,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-18,21.113824,51,0.413997,416.311514,313,1.330069,16682,17195,15446.717053,7490.934158,...,156.954898,2.122852,1.361268,0.639252,0.332954,0.021228,0.021006,0.412789,0.582972,47373.398942
2021-09-19,40.032766,66,0.606557,366.559556,309,1.186277,14052,15320,13368.003124,6226.174704,...,-3.806675,-2.190376,-0.907229,-0.435480,0.319772,-0.021900,-0.022147,0.582972,0.969921,47350.740700
2021-09-20,275.415063,103,2.673933,304.137201,360,0.844826,23068,22064,40580.947165,16215.253160,...,-85.108841,-8.966624,-2.132911,-9.259215,0.483109,-0.089742,-0.093944,0.969921,0.533894,46561.907572


In [11]:
df.drop([
'average_cap',
 'delta_cap',
 'thermo_cap',
'stablecoins_ratio',
'stablecoins_ratio_usd',
'average_sa_cdd',
'binary_cdd',
'mcda',
'scda',
'range_0d_1d',
 'range_1d_1w',
 'range_1w_1m',
 'range_1m_3m',
 'range_3m_6m',
 'range_6m_12m',
 'range_12m_18m',
 'range_18m_2y',
 'range_2y_3y',
 'range_3y_5y',
 'range_5y_7y',
 'range_7y_10y',
 'range_10y_inf',
 'range_0d_1d_percent',
 'range_1d_1w_percent',
 'range_1w_1m_percent',
 'range_1m_3m_percent',
 'range_3m_6m_percent',
 'range_6m_12m_percent',
 'range_12m_18m_percent',
 'range_18m_2y_percent',
 'range_2y_3y_percent',
 'range_3y_5y_percent',
 'range_5y_7y_percent',
 'range_7y_10y_percent',
 'range_10y_inf_percent'], axis=1, inplace=True)

In [12]:
df_sent=pd.read_csv("final_twitter_sentiments.csv")
df_sent

,Datetime,BTC,USD,BTC_change,USD_change,date,datetime4h,datetime1m,datetime5m,datetimeday,datetime15m,datetime1h,datetime12h
0,2021-10-17 21:34:18+00:00,645,38674244,645,38674244,2021-10-17,2021-10-17 20:00:00,2021-10-17 21:34:00,2021-10-17 21:30:00,2021-10-17,2021-10-17 21:30:00,2021-10-17 21:00:00,2021-10-17 12:00:00
1,2021-10-15 20:59:41+00:00,341,21397789,341,21397789,2021-10-15,2021-10-15 20:00:00,2021-10-15 20:59:00,2021-10-15 20:55:00,2021-10-15,2021-10-15 20:45:00,2021-10-15 20:00:00,2021-10-15 12:00:00
2,2021-10-15 17:59:04+00:00,500,30861741,500,30861741,2021-10-15,2021-10-15 16:00:00,2021-10-15 17:59:00,2021-10-15 17:55:00,2021-10-15,2021-10-15 17:45:00,2021-10-15 17:00:00,2021-10-15 12:00:00
3,2021-10-15 17:11:18+00:00,881,54535823,881,54535823,2021-10-15,2021-10-15 16:00:00,2021-10-15 17:11:00,2021-10-15 17:10:00,2021-10-15,2021-10-15 17:00:00,2021-10-15 17:00:00,2021-10-15 12:00:00
4,2021-10-14 19:10:30+00:00,839,48322579,839,48322579,2021-10-14,2021-10-14 16:00:00,2021-10-14 19:10:00,2021-10-14 19:10:00,2021-10-14,2021-10-14 19:00:00,2021-10-14 19:00:00,2021-10-14 12:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7428,2018-09-21 05:06:41+00:00,949,6221999,-949,-6221999,2018-09-21,2018-09-21 04:00:00,2018-09-21 05:06:00,2018-09-21 05:05:00,2018-09-21,2018-09-21 05:00:00,2018-09-21 05:00:00,2018-09-21 00:00:00
7429,2018-09-21 05:06:41+00:00,1000,6555335,-1000,-6555335,2018-09-21,2018-09-21 04:00:00,2018-09-21 05:06:00,2018-09-21 05:05:00,2018-09-21,2018-09-21 05:00:00,2018-09-21 05:00:00,2018-09-21 00:00:00
7430,2018-09-21 02:32:07+00:00,1500,9769703,-1500,-9769703,2018-09-21,2018-09-21 00:00:00,2018-09-21 02:32:00,2018-09-21 02:30:00,2018-09-21,2018-09-21 02:30:00,2018-09-21 02:00:00,2018-09-21 00:00:00
7431,2018-09-20 22:23:49+00:00,1229,7969815,-1229,-7969815,2018-09-20,2018-09-20 20:00:00,2018-09-20 22:23:00,2018-09-20 22:20:00,2018-09-20,2018-09-20 22:15:00,2018-09-20 22:00:00,2018-09-20 12:00:00


In [13]:
df_sent=df_sent[["date","BTC","USD","BTC_change","USD_change"]]
df_sent['date'] = pd.to_datetime(df_sent['date'])
df_sent

,date,BTC,USD,BTC_change,USD_change
0,2021-10-17,645,38674244,645,38674244
1,2021-10-15,341,21397789,341,21397789
2,2021-10-15,500,30861741,500,30861741
3,2021-10-15,881,54535823,881,54535823
4,2021-10-14,839,48322579,839,48322579
...,...,...,...,...,...
7428,2018-09-21,949,6221999,-949,-6221999
7429,2018-09-21,1000,6555335,-1000,-6555335
7430,2018-09-21,1500,9769703,-1500,-9769703
7431,2018-09-20,1229,7969815,-1229,-7969815


In [14]:
df_sent2=df_sent.groupby(['date'],as_index=False).agg({'BTC': 'sum','USD':'sum','BTC_change':'sum','USD_change':'sum'})
df_sent2=df_sent2.set_index("date")
df_sent2

,BTC,USD,BTC_change,USD_change
date,,,,
2018-09-18,1996,12557200,1996,12557200
2018-09-19,5247,33374272,2247,14258730
2018-09-20,3144,20339627,686,4399997
2018-09-21,21847,145668621,11337,76181865
2018-09-22,5360,36022711,5360,36022711
...,...,...,...,...
2021-10-13,7656,431112884,2916,162909088
2021-10-14,3925,226553480,1625,93516882
2021-10-15,12077,733976028,-8633,-520385322


In [15]:
comb2=pd.merge(df,df_sent2, how='left', left_index=True, right_index=True)
comb2

,etom_flow_total,etom_transactions_count_flow,etom_flow_mean,mtoe_flow_total,mtoe_transactions_count_flow,mtoe_flow_mean,exchange_addresses_count_inflow,exchange_addresses_count_outflow,exchange_inflow_total,exchange_inflow_top10,...,HL_sprd,CO_sprd,log_returns,vol_current,vol_future,ema10,BTC,USD,BTC_change,USD_change
date,,,,,,,,,,,,,,,,,,,,,
2016-01-01,239.315793,117,2.045434,245.037136,51,4.804650,17861,8225,23829.684650,15442.076341,...,0.292472,0.011277,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-02,128.735315,93,1.384251,237.954298,62,3.837973,18755,9181,11099.867349,5736.771814,...,0.257968,-0.005073,-0.001550,NaN,0.080802,NaN,NaN,NaN,NaN,NaN
2016-01-03,213.788376,121,1.766846,151.075527,51,2.962265,20945,9515,13295.729379,7174.527336,...,0.319309,-0.011390,-0.007531,0.080802,0.195726,NaN,NaN,NaN,NaN,NaN
2016-01-04,445.146297,141,3.157066,345.013905,65,5.307906,18019,11310,21875.941857,11665.436371,...,0.273074,0.008310,0.006957,0.195726,0.088823,NaN,NaN,NaN,NaN,NaN
2016-01-05,506.206706,122,4.149235,263.934619,76,3.472824,18473,10344,19891.738077,11643.719982,...,0.257792,0.001443,0.000382,0.088823,0.139956,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-18,21.113824,51,0.413997,416.311514,313,1.330069,16682,17195,15446.717053,7490.934158,...,0.332954,0.021228,0.021006,0.412789,0.582972,47373.398942,1062.0,50968748.0,-1062.0,-50968748.0
2021-09-19,40.032766,66,0.606557,366.559556,309,1.186277,14052,15320,13368.003124,6226.174704,...,0.319772,-0.021900,-0.022147,0.582972,0.969921,47350.740700,600.0,28366734.0,-600.0,-28366734.0
2021-09-20,275.415063,103,2.673933,304.137201,360,0.844826,23068,22064,40580.947165,16215.253160,...,0.483109,-0.089742,-0.093944,0.969921,0.533894,46561.907572,7906.0,346607470.0,4600.0,200197290.0


In [16]:
comb2=comb2.reset_index()
df=comb2
df

,date,etom_flow_total,etom_transactions_count_flow,etom_flow_mean,mtoe_flow_total,mtoe_transactions_count_flow,mtoe_flow_mean,exchange_addresses_count_inflow,exchange_addresses_count_outflow,exchange_inflow_total,...,HL_sprd,CO_sprd,log_returns,vol_current,vol_future,ema10,BTC,USD,BTC_change,USD_change
0,2016-01-01,239.315793,117,2.045434,245.037136,51,4.804650,17861,8225,23829.684650,...,0.292472,0.011277,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-02,128.735315,93,1.384251,237.954298,62,3.837973,18755,9181,11099.867349,...,0.257968,-0.005073,-0.001550,NaN,0.080802,NaN,NaN,NaN,NaN,NaN
2,2016-01-03,213.788376,121,1.766846,151.075527,51,2.962265,20945,9515,13295.729379,...,0.319309,-0.011390,-0.007531,0.080802,0.195726,NaN,NaN,NaN,NaN,NaN
3,2016-01-04,445.146297,141,3.157066,345.013905,65,5.307906,18019,11310,21875.941857,...,0.273074,0.008310,0.006957,0.195726,0.088823,NaN,NaN,NaN,NaN,NaN
4,2016-01-05,506.206706,122,4.149235,263.934619,76,3.472824,18473,10344,19891.738077,...,0.257792,0.001443,0.000382,0.088823,0.139956,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2087,2021-09-18,21.113824,51,0.413997,416.311514,313,1.330069,16682,17195,15446.717053,...,0.332954,0.021228,0.021006,0.412789,0.582972,47373.398942,1062.0,50968748.0,-1062.0,-50968748.0
2088,2021-09-19,40.032766,66,0.606557,366.559556,309,1.186277,14052,15320,13368.003124,...,0.319772,-0.021900,-0.022147,0.582972,0.969921,47350.740700,600.0,28366734.0,-600.0,-28366734.0
2089,2021-09-20,275.415063,103,2.673933,304.137201,360,0.844826,23068,22064,40580.947165,...,0.483109,-0.089742,-0.093944,0.969921,0.533894,46561.907572,7906.0,346607470.0,4600.0,200197290.0
2090,2021-09-21,91.547305,108,0.847660,174.964510,351,0.498474,21263,19524,34547.689220,...,0.461613,-0.052905,-0.054423,0.533894,1.643500,45502.261187,3962.0,168897153.0,426.0,19556329.0


In [17]:
df['funding_rates'].fillna(0, inplace=True)
df['ema10'].fillna(method='backfill', inplace=True)
df.fillna(1, inplace=True)

In [18]:
df['BTCminus'] = 1
df.BTCminus.loc[df['BTC_change']<0] = df['BTC_change'].abs()


df['BTCplus'] = 1
df.BTCplus.loc[df['BTC_change']>0] = df['BTC_change']


df['USDminus'] = 1
df.USDminus.loc[df['USD_change']<0] = df['USD_change'].abs()


df['USDplus'] = 1
df.USDplus.loc[df['USD_change']>0] = df['USD_change']


In [19]:
df['exchange_minus'] = 1
df.exchange_minus.loc[df['exchange_netflow_total']<0] = df['exchange_netflow_total'].abs() +1

df['exchange_plus'] = 1
df.exchange_plus.loc[df['exchange_netflow_total']>0] = df['exchange_netflow_total']

df['miner_minus'] = 1
df.miner_minus.loc[df['miner_netflow_total']<0] = df['miner_netflow_total'].abs()  +1

df['miner_plus'] = 1
df.miner_plus.loc[df['miner_netflow_total']>0] = df['miner_netflow_total']

In [20]:
df.drop(['exchange_netflow_total','miner_netflow_total'],axis=1,inplace=True)

In [21]:
df.drop(['BTC','USD','BTC_change','USD_change'],axis=1,inplace=True)
df

,date,etom_flow_total,etom_transactions_count_flow,etom_flow_mean,mtoe_flow_total,mtoe_transactions_count_flow,mtoe_flow_mean,exchange_addresses_count_inflow,exchange_addresses_count_outflow,exchange_inflow_total,...,vol_future,ema10,BTCminus,BTCplus,USDminus,USDplus,exchange_minus,exchange_plus,miner_minus,miner_plus
0,2016-01-01,239.315793,117,2.045434,245.037136,51,4.804650,17861,8225,23829.684650,...,1.000000,440.653344,1.0,1.0,1.0,1.0,1.000000,9759.440821,1.000000,1693.817533
1,2016-01-02,128.735315,93,1.384251,237.954298,62,3.837973,18755,9181,11099.867349,...,0.080802,440.653344,1.0,1.0,1.0,1.0,1736.356273,1.000000,1.000000,1303.000408
2,2016-01-03,213.788376,121,1.766846,151.075527,51,2.962265,20945,9515,13295.729379,...,0.195726,440.653344,1.0,1.0,1.0,1.0,333.930916,1.000000,1.000000,1386.907567
3,2016-01-04,445.146297,141,3.157066,345.013905,65,5.307906,18019,11310,21875.941857,...,0.088823,440.653344,1.0,1.0,1.0,1.0,1.000000,963.778703,1.000000,3541.923439
4,2016-01-05,506.206706,122,4.149235,263.934619,76,3.472824,18473,10344,19891.738077,...,0.139956,440.653344,1.0,1.0,1.0,1.0,191.669303,1.000000,4773.476726,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2087,2021-09-18,21.113824,51,0.413997,416.311514,313,1.330069,16682,17195,15446.717053,...,0.582972,47373.398942,1062.0,1.0,50968748.0,1.0,1.000000,379.994037,1.000000,156.954898
2088,2021-09-19,40.032766,66,0.606557,366.559556,309,1.186277,14052,15320,13368.003124,...,0.969921,47350.740700,600.0,1.0,28366734.0,1.0,1337.217043,1.000000,4.806675,1.000000
2089,2021-09-20,275.415063,103,2.673933,304.137201,360,0.844826,23068,22064,40580.947165,...,0.533894,46561.907572,1.0,4600.0,1.0,200197290.0,1.000000,5155.599783,86.108841,1.000000
2090,2021-09-21,91.547305,108,0.847660,174.964510,351,0.498474,21263,19524,34547.689220,...,1.643500,45502.261187,1.0,426.0,1.0,19556329.0,1.000000,2334.305389,87.106321,1.000000


In [22]:
all_col=[
'etom_flow_total',
'mtoe_flow_total',
    
'etom_transactions_count_flow',
'mtoe_transactions_count_flow',
    
'exchange_addresses_count_inflow',
'exchange_addresses_count_outflow',
'miner_addresses_count_inflow',
'miner_addresses_count_outflow',
    
'exchange_transactions_count_inflow',
'exchange_transactions_count_outflow',
'miner_transactions_count_inflow',
'miner_transactions_count_outflow',   
    
'exchange_inflow_total',
'exchange_inflow_top10',       
'miner_inflow_total',
'miner_inflow_top10',               
'miner_outflow_total',
'miner_outflow_top10',


'market_cap',
'realized_cap',
'long_liquidations',
'short_liquidations',
'long_liquidations_usd',
'short_liquidations_usd',

'open','high','low','close','volume',
'open_interest',
'hashrate',    
'cdd',
'sca', 
'ema10',


'taker_buy_volume',         
'taker_sell_volume', 

'BTCminus',
'USDminus',
'BTCplus',
'USDplus',



'exchange_outflow_total',
'exchange_outflow_top10',
    
'exchange_minus',
'miner_minus',
'exchange_plus',
'miner_plus']

In [23]:
df[all_col]=df[all_col].apply(np.log)
df

,date,etom_flow_total,etom_transactions_count_flow,etom_flow_mean,mtoe_flow_total,mtoe_transactions_count_flow,mtoe_flow_mean,exchange_addresses_count_inflow,exchange_addresses_count_outflow,exchange_inflow_total,...,vol_future,ema10,BTCminus,BTCplus,USDminus,USDplus,exchange_minus,exchange_plus,miner_minus,miner_plus
0,2016-01-01,5.477784,4.762174,2.045434,5.501410,3.931826,4.804650,9.790375,9.014934,10.078687,...,1.000000,6.088258,0.000000,0.000000,0.000000,0.000000,0.000000,9.185990,0.000000,7.434740
1,2016-01-02,4.857758,4.532599,1.384251,5.472079,4.127134,3.837973,9.839216,9.124891,9.314688,...,0.080802,6.088258,0.000000,0.000000,0.000000,0.000000,7.459544,0.000000,0.000000,7.172425
2,2016-01-03,5.364987,4.795791,1.766846,5.017780,3.931826,2.962265,9.949655,9.160625,9.495198,...,0.195726,6.088258,0.000000,0.000000,0.000000,0.000000,5.810934,0.000000,0.000000,7.234832
3,2016-01-04,6.098403,4.948760,3.157066,5.843585,4.174387,5.307906,9.799182,9.333443,9.993143,...,0.088823,6.088258,0.000000,0.000000,0.000000,0.000000,0.000000,6.870862,0.000000,8.172425
4,2016-01-05,6.226945,4.804021,4.149235,5.575701,4.330733,3.472824,9.824065,9.244162,9.898060,...,0.139956,6.088258,0.000000,0.000000,0.000000,0.000000,5.255772,0.000000,8.470830,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2087,2021-09-18,3.049928,3.931826,0.413997,6.031434,5.746203,1.330069,9.722086,9.752374,9.645152,...,0.582972,10.765816,6.967909,0.000000,17.746723,0.000000,0.000000,5.940156,0.000000,5.055958
2088,2021-09-19,3.689698,4.189655,0.606557,5.904161,5.733341,1.186277,9.550520,9.636914,9.500619,...,0.969921,10.765338,6.396930,0.000000,17.160728,0.000000,7.198346,0.000000,1.570006,0.000000
2089,2021-09-20,5.618279,4.634729,2.673933,5.717479,5.886104,0.844826,10.046202,10.001703,10.611054,...,0.533894,10.748538,0.000000,8.433812,0.000000,19.114814,0.000000,8.547839,4.455612,0.000000
2090,2021-09-21,4.516856,4.682131,0.847660,5.164583,5.860786,0.498474,9.964724,9.879400,10.450096,...,1.643500,10.725517,0.000000,6.054439,0.000000,16.788810,0.000000,7.755470,4.467129,0.000000


In [24]:
df2=df

In [25]:
df2['velocity_supply_total']=df2['velocity_supply_total'].apply(np.log)
df2['mca']=df2['mca'].apply(np.log)

df2['miner_outflow_mean_ma7']=df2['miner_outflow_mean_ma7'].apply(np.log)
df2['exchange_outflow_mean_ma7']=df2['exchange_outflow_mean_ma7'].apply(np.log)


In [26]:
df2['etom_flow_mean']=df2['etom_flow_mean']+1
df2['etom_flow_mean']=df2['etom_flow_mean'].apply(np.log)

df2['mtoe_flow_mean']=df2['mtoe_flow_mean']+1
df2['mtoe_flow_mean']=df2['mtoe_flow_mean'].apply(np.log)



df2['sa_cdd']=df2['sa_cdd']**(1/4)


df2['miner_inflow_mean']=df2['miner_inflow_mean'].apply(np.cbrt)
df2['exchange_inflow_mean']=df2['exchange_inflow_mean'].apply(np.cbrt)



df2['miner_inflow_mean_ma7']=df2['miner_inflow_mean_ma7'].apply(np.sqrt)
df2['exchange_inflow_mean_ma7']=df2['exchange_inflow_mean_ma7'].apply(np.sqrt)


df2['mvrv']=df2['mvrv'].apply(np.sqrt)


df2['miner_outflow_mean']=df2['miner_outflow_mean'].apply(np.cbrt)
#df2['miner_outflow_mean_ma7']=df2['miner_outflow_mean_ma7']**(1/4)

df2['exchange_outflow_mean']=df2['exchange_outflow_mean'].apply(np.cbrt)
#df2['exchange_outflow_mean_ma7']=df2['exchange_outflow_mean_ma7']**(1/4)






In [27]:


to_drop2=[ 
'realized_cap',
'sa_cdd',    
'taker_buy_sell_ratio',
'mca','hashrate','velocity_supply_total','vol_current','returns'
'high_pct_change',
'low_pct_change'
]



df2.drop(to_drop2, axis=1, inplace=True)
df2.drop(df.tail(1).index,inplace=True) # drop last n rows
df2.drop(df.head(1).index,inplace=True) # drop first n rows

df2['vol_future']=df2['vol_future']**(1/4)
df2.to_csv("dataset.csv",index=False)